#### 문제 4-1 : 카페 메뉴 도구(Tool) 호출 체인 구현 ( LangChain 사용)


In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:4])

sk
18
tvly


In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [3]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 3개의 결과 반환)
#web_search = TavilySearchResults(max_results=2)
web_search = TavilySearchResults(
    # 검색 결과의 최대 개수를 3개로 제한합니다.
    max_results=3,

    # 검색 깊이를 "고급(advanced)"으로 설정합니다.
    # 일반(basic) 검색보다 더 깊이 있고 포괄적인 결과를 제공해 복잡한 질문에 더 적합합니다.
    search_depth="advanced",

    # 검색 결과에 질문에 대한 요약된 답변을 포함시킵니다.
    # 이를 통해 에이전트가 별도의 추론 없이 핵심 정보를 빠르게 얻을 수 있습니다.
    include_answer=True,

    # 검색 결과에 원본 웹페이지의 전체 콘텐츠를 포함시킵니다.
    # 답변에 대한 더 자세한 맥락이나 원문이 필요할 때 유용합니다.
    include_raw_content=True,

    # 검색 결과에 관련된 이미지를 포함시킵니다.
    # 시각적인 정보가 필요한 경우(예: 특정 장소, 제품 등) 유용합니다.
    include_images=True,
)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력 [dict, dict]
for result in search_results:
    print(type(result))
    print(result.keys())  
    print(result['score'], result['url'])
    print("-" * 100)  

C:\Users\user\AppData\Local\Temp\ipykernel_18640\3260950074.py:8: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search = TavilySearchResults(


<class 'dict'>
dict_keys(['title', 'url', 'content', 'score', 'raw_content'])
0.9317675 https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/
----------------------------------------------------------------------------------------------------
<class 'dict'>
dict_keys(['title', 'url', 'content', 'score', 'raw_content'])
0.93161833 https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80
----------------------------------------------------------------------------------------------------
<class 'dict'>
dict_keys(['title', 'url', 'content', 'score', 'raw_content'])
0.9061314 https://m.blog.naver.com/wineislikeacat/223096696241
---------------------

In [4]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search_results_json
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. Input '
 'should be a search query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}
----------------------------------------------------------------------------------------------------

`(2) 도구(tool) 호출하기`

In [5]:
from langchain_openai import ChatOpenAI
# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# from langchain_upstage import ChatUpstage
# llm = ChatUpstage(
#         model="solar-pro",
#         base_url="https://api.upstage.ai/v1",
#         temperature=0.5
#     )
print(llm.model_name)

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])
print(type(llm_with_tools))

gpt-4o-mini
<class 'langchain_core.runnables.base.RunnableBinding'>


In [6]:
from pprint import pprint

# === 1. 도구 호출이 필요 없는 일반 대화 테스트 ===
query = "안녕하세요?"
ai_msg = llm_with_tools.invoke(query)

print("\n=== 일반 대화 응답 ===")
print("AIMessage 타입:", type(ai_msg))
print("-" * 100)
print("전체 출력:")
pprint(ai_msg)
print("-" * 100)
print("content:", ai_msg.content)
print("-" * 100)
print("tool_calls:", ai_msg.tool_calls)  # []
print("=" * 100)

# === 2. 도구 호출이 필요한 질문 테스트 ===
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

print("\n=== 도구 호출 테스트 ===")
print("AIMessage 타입:", type(ai_msg))
print("-" * 100)
print("전체 출력:")
pprint(ai_msg)
print("-" * 100)
print("content:", ai_msg.content)
print("-" * 100)
print("tool_calls:", ai_msg.tool_calls)
print("=" * 100)

# === 3. 첫 번째 tool_call 확인 ===
if ai_msg.tool_calls:
    tool_call = ai_msg.tool_calls[0]
    print("\n=== 첫 번째 tool_call ===")
    pprint(tool_call)
    print("=" * 100)
else:
    print("\n[!] tool_calls가 비어 있습니다.")


=== 일반 대화 응답 ===
AIMessage 타입: <class 'langchain_core.messages.ai.AIMessage'>
----------------------------------------------------------------------------------------------------
전체 출력:
AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 82, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CL3vxFWj9itT5NvYHntrYaM9tAqJ5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2ee46cd1-58ed-4c76-a37f-46bf109867f0-0', usage_metadata={'input_tokens': 82, 'output_tokens': 11, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 

`(3) 도구(tool) 실행하기`

In [ ]:
from pprint import pprint

# === 단일 tool_call 실행 ===
print("\n=== 단일 tool_call 실행 ===")
tool_call = ai_msg.tool_calls[0]   # 첫 번째 tool_call 선택
tool_message = web_search.invoke(tool_call)

print("ToolMessage 타입:", type(tool_message))
print("\n=== ToolMessage 주요 속성 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")

print("\n=== ToolMessage.content 타입 ===")
pprint(tool_message.content)

print("\n=== ToolMessage.tool_call_id ===")
print("\n=== ToolMessage.name ===")


print("=" * 100)

# === 전체 tool_calls batch 실행 ===
print("\n=== batch 실행 (tool_calls가 여러 개인 경우) ===")
tool_messages = web_search.batch(ai_msg.tool_calls)

print("Batch 실행 결과 개수:", len(tool_messages))
print("-" * 100)

for i, tm in enumerate(tool_messages, 1):
    print(f"\n--- ToolMessage #{i} ---")
    pprint(tm.content)
    print("-" * 100)

`(4) @chain 데코레이터를 사용하여 RunnableLambda로 생성하기`

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_core.tools import tool

# 1. 메뉴 데이터 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()
print(f"문서 개수: {len(documents)}")

# 2. 문서 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.split_documents(documents)
print(f"분할된 문서 개수: {len(docs)}")

# 3. 임베딩 & 벡터 DB 생성
embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")
cafe_db = FAISS.from_documents(docs, embedding=embeddings_model)

# 4. 로컬 저장
cafe_db.save_local("../db/cafe_db")
print("cafe_db 벡터 인덱스 생성 및 저장 완료")

# 5. 검색용 Tool 정의
retriever = cafe_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

@tool
def db_search_cafe_func(query: str):
    """로컬 카페 메뉴 DB에서 관련 정보를 검색합니다."""
    results = retriever.get_relevant_documents(query)
    return [doc.page_content for doc in results]

문서 개수: 1
분할된 문서 개수: 10
cafe_db 벡터 인덱스 생성 및 저장 완료


In [16]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from pprint import pprint

# === 1. 프롬프트 템플릿 ===
today = datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate([
    ("system", f"""
    You are a helpful AI assistant for a cafe.
    Today's date is {today}.
    Your job is to answer questions about cafe menu, coffee history, and coffee trends
    by calling the appropriate tools.
    """),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# === 2. db_search_cafe_func (반환값 수정) ===
@tool
def db_search_cafe_func(query: str):
    """로컬 카페 메뉴 DB에서 관련 정보를 검색합니다."""
    results = retriever.get_relevant_documents(query)
    # 문자열 하나로 합쳐서 반환 (list[str] → str)
    return "\n\n".join([doc.page_content for doc in results])

# === 3. LLM + 도구 바인딩 ===
llm_with_tools = llm.bind_tools(tools=[db_search_cafe_func])
llm_chain = prompt | llm_with_tools

# === 4. 체인 정의 ===
@chain
def cafe_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    
    # 1차 LLM 호출 (어떤 도구 쓸지 결정)
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg:\n", ai_msg)
    print("-" * 100)

    # 도구 실행
    tool_msgs = db_search_cafe_func.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs:\n", tool_msgs)
    print("-" * 100)

    # 2차 LLM 호출 (도구 실행 결과 반영해서 최종 답변 생성)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# === 5. 체인 실행 ===
query = "아메리카노의 가격과 특징은 무엇인가요?"
response = cafe_chain.invoke(query)

# === 6. 응답 출력 ===
pprint(response.content)

ai_msg:
 content='' additional_kwargs={'tool_calls': [{'id': 'call_QVSXOnmLPXGLoLkxgDdswKQH', 'function': {'arguments': '{"query":"아메리카노"}', 'name': 'db_search_cafe_func'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 116, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CL4O08Fecgz8SfyCQnD3weALLsbDN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--f26d295b-8e5d-4ea0-b3d8-5039618bb75b-0' tool_calls=[{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_QVSXOnmLPXGLoLkxgDdswKQH', 'type': 'tool_call'}] usage_metadata={'input_tokens': 116, 'output_tokens': 21, 'total_tokens': 137, 'input_token_de

In [17]:
from langchain_community.tools import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# === 7. 추가 도구 정의 ===

# 웹 검색 (Tavily)
@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다 (예: 커피 트렌드, 카페 위치)."""
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    formatted_docs = "\n---\n".join([
        f"<Document href='{doc['url']}'>\n{doc['content']}\n</Document>"
        for doc in docs
    ])
    return formatted_docs if formatted_docs else "관련 정보를 찾을 수 없습니다."

# 위키 요약 (Wikipedia)
wiki_api = WikipediaAPIWrapper(lang="ko")
wiki = WikipediaQueryRun(api_wrapper=wiki_api)

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 일반 지식(예: 커피 역사, 제조법)을 검색합니다."""
    return wiki.run(query)

# === 8. LLM + 모든 도구 바인딩 ===
llm_with_tools = llm.bind_tools(
    tools=[db_search_cafe_func, tavily_search_func, wiki_summary]
)
llm_chain = prompt | llm_with_tools

# === 9. 통합 체인 정의 ===
@chain
def cafe_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}

    # 1차 LLM 호출 (도구 선택)
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg:\n", ai_msg)
    print("-" * 100)

    # 도구 실행
    tool_msgs = []
    for tc in ai_msg.tool_calls:
        if tc["name"] == "db_search_cafe_func":
            tool_msgs.extend(db_search_cafe_func.batch([tc], config=config))
        elif tc["name"] == "tavily_search_func":
            tool_msgs.extend(tavily_search_func.batch([tc], config=config))
        elif tc["name"] == "wiki_summary":
            tool_msgs.extend(wiki_summary.batch([tc], config=config))

    print("tool_msgs:\n", tool_msgs)
    print("-" * 100)

    # 2차 LLM 호출 (도구 실행 결과 반영해서 최종 답변 생성)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# === 10. 테스트 실행 ===
query = "커피의 역사에 대해 알려줘"
response = cafe_chain.invoke(query)

pprint(response.content)

ai_msg:
 content='' additional_kwargs={'tool_calls': [{'id': 'call_vJzUAdvyToVN1NggYjVCKCl4', 'function': {'arguments': '{"query":"커피의 역사"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 181, 'total_tokens': 199, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CL4UkyykWlj8Kzi9TnjAcPWpasbh8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--30a94a59-be7e-4da0-91c4-5894855a6b9c-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '커피의 역사'}, 'id': 'call_vJzUAdvyToVN1NggYjVCKCl4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 181, 'output_tokens': 18, 'total_tokens': 199, 'input_token_details': {'au

In [18]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다."""
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f"<Document href='{doc['url']}'>\n{doc['content']}\n</Document>"
        for doc in docs
    ])
    return formatted_docs if formatted_docs else "관련 정보를 찾을 수 없습니다."


# === LLM + 3개 도구 바인딩 ===
llm_with_tools = llm.bind_tools(tools=[db_search_cafe_func, wiki_summary, tavily_search_func])
llm_chain = prompt | llm_with_tools

# === 통합 체인 정의 ===
@chain
def cafe_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}

    # 1차 LLM 호출 (어떤 도구 쓸지 결정)
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg:\n", ai_msg)
    print("-" * 100)

    # 도구 실행
    tool_msgs = []
    if ai_msg.tool_calls:
        for tool_call in ai_msg.tool_calls:
            if tool_call["name"] == "db_search_cafe_func":
                tool_msgs.extend(db_search_cafe_func.batch([tool_call], config=config))
            elif tool_call["name"] == "wiki_summary":
                tool_msgs.extend(wiki_summary.batch([tool_call], config=config))
            elif tool_call["name"] == "tavily_search_func":
                tool_msgs.extend(tavily_search_func.batch([tool_call], config=config))

    print("tool_msgs:\n", tool_msgs)
    print("-" * 100)

    # 2차 LLM 호출 (도구 실행 결과 반영해서 최종 답변 생성)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# === 테스트 실행 ===
print("\n[DB 검색] =====================")
response1 = cafe_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")
print(response1.content)

print("\n[위키 검색] =====================")
response2 = cafe_chain.invoke("커피의 역사를 알려줘")
print(response2.content)

print("\n[웹 검색] =====================")
response3 = cafe_chain.invoke("최근에 가장 인기있는 파스타 식당은 어디야?")
print(response3.content)


[DB 검색] =====================
ai_msg:
 content='' additional_kwargs={'tool_calls': [{'id': 'call_TZr9Q6aeP9C14btdNQAJN1os', 'function': {'arguments': '{"query":"아메리카노"}', 'name': 'db_search_cafe_func'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 175, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CL4WFo2MtaQxfyM0UgGMWZQtav9kN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--3a9e9f3d-f50b-4bb4-a92a-ff3636987203-0' tool_calls=[{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_TZr9Q6aeP9C14btdNQAJN1os', 'type': 'tool_call'}] usage_metadata={'input_tokens': 175, 'output_tokens': 21, 'tota